In [21]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np


There will be three main parts of the code: dataset preparation, model training, and generating prediction. The boiler plate code of this architecture is following:


In [22]:
def dataset_preparation():
    pass 
def create_model():
    pass
def generate_text():
    pass


Dataset preparation

In [23]:
def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization	
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences 
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words


In [24]:
def create_model(predictors, label, max_sequence_len, total_words):
	
	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	# model.add(Dropout(0.2))
	model.add(LSTM(100))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
	model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
	print(model.summary())
	return model 



In [25]:
def generate_text(seed_text, next_words, max_sequence_len):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict_classes(token_list, verbose=0)
		
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text


In [26]:
data = open('data.txt').read()

predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
print(generate_text("we naughty", 3, max_sequence_len))


Epoch 1/100
48/48 [==============================] - 2s 33ms/step - loss: 3.7621 - acc: 0.0000e+00
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7586 - acc: 0.0833
Epoch 3/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7558 - acc: 0.0625
Epoch 4/100
48/48 [==============================] - 0s 985us/step - loss: 3.7532 - acc: 0.0625
Epoch 5/100
32/48 [===================>..........] - ETA: 0s - loss: 3.7491 - acc: 0.0625

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


48/48 [==============================] - 0s 2ms/step - loss: 3.7493 - acc: 0.0625
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7447 - acc: 0.0625
Epoch 7/100
48/48 [==============================] - ETA: 0s - loss: 3.7359 - acc: 0.093 - 0s 1ms/step - loss: 3.7394 - acc: 0.0625
Epoch 8/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7299 - acc: 0.0625
Epoch 9/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7167 - acc: 0.0625
Epoch 10/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7034 - acc: 0.0625
Epoch 11/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6779 - acc: 0.0625
Epoch 12/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6472 - acc: 0.0625
Epoch 13/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6223 - acc: 0.0625
Epoch 14/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6078 - acc: 0.0625
Epoch 15/100
48/48 [=========

48/48 [==============================] - 0s 1ms/step - loss: 1.9866 - acc: 0.4375
Epoch 92/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9433 - acc: 0.4792
Epoch 93/100
48/48 [==============================] - 0s 1ms/step - loss: 1.9495 - acc: 0.4792
Epoch 94/100
48/48 [==============================] - 0s 990us/step - loss: 1.9114 - acc: 0.4792
Epoch 95/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8931 - acc: 0.4583
Epoch 96/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8817 - acc: 0.5000
Epoch 97/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8507 - acc: 0.4583
Epoch 98/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8421 - acc: 0.4792
Epoch 99/100
48/48 [==============================] - 0s 1ms/step - loss: 1.8043 - acc: 0.5417
Epoch 100/100
48/48 [==============================] - 0s 1ms/step - loss: 1.7789 - acc: 0.5625
____________________________________________________________

In [27]:
print(generate_text("cat and", 3, max_sequence_len))


cat and lost lost put


In [16]:
data = """The cat and her kittens
They put on their mittens,
To eat a Christmas pie.
The poor little kittens
They lost their mittens,
And then they began to cry.
O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens."
"If it be so, ye shall not go,
For ye are naughty kittens."""
predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
print(generate_text("we naughty", 3, max_sequence_len))


Epoch 1/100
48/48 [==============================] - 1s 28ms/step - loss: 3.7619 - acc: 0.0417
Epoch 2/100
48/48 [==============================] - 0s 768us/step - loss: 3.7581 - acc: 0.0625
Epoch 3/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7550 - acc: 0.0625
Epoch 4/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7515 - acc: 0.0625
Epoch 5/100
32/48 [===================>..........] - ETA: 0s - loss: 3.7427 - acc: 0.0625

/usr/local/lib/python3.5/dist-packages/keras/callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


48/48 [==============================] - 0s 1ms/step - loss: 3.7475 - acc: 0.0625
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7407 - acc: 0.0625
Epoch 7/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7324 - acc: 0.0625
Epoch 8/100
48/48 [==============================] - 0s 942us/step - loss: 3.7194 - acc: 0.0625
Epoch 9/100
48/48 [==============================] - 0s 1ms/step - loss: 3.7084 - acc: 0.0625
Epoch 10/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6833 - acc: 0.0625
Epoch 11/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6606 - acc: 0.0625
Epoch 12/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6355 - acc: 0.0625
Epoch 13/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6279 - acc: 0.0625
Epoch 14/100
48/48 [==============================] - 0s 1ms/step - loss: 3.6392 - acc: 0.0625
Epoch 15/100
48/48 [==============================] - 0s 1ms/step

In [17]:
print(generate_text("cat and", 3, max_sequence_len))


cat and cannot it to
